In [6]:
import ee
import pandas as pd
import geopandas as gpd
import numpy as np
from datetime import datetime, timedelta
from pathlib import Path

print("=" * 60)
print("GPM PRECIPITATION FEATURE EXTRACTION")
print("=" * 60)

# Initialize Earth Engine
ee.Initialize(project="kolkata-flood-mapping")

# Load ward boundaries
WARDS_FILE = Path('../data/wards/kmc_wards_gee_ready.geojson')
wards_gdf = gpd.read_file(WARDS_FILE)

# FIX: Strip newline characters from ward IDs
wards_gdf['WARD'] = wards_gdf['WARD'].str.strip()

print(f"\n✅ Loaded {len(wards_gdf)} wards from local file")

# Convert to projected CRS, calculate centroids, then back to WGS84
wards_gdf = wards_gdf.to_crs('EPSG:32645')  # UTM Zone 45N
wards_gdf['geometry'] = wards_gdf.geometry.centroid
wards_gdf = wards_gdf.to_crs('EPSG:4326')  # Back to WGS84

# Create ward coordinates dictionary
ward_coords = {}
for idx, row in wards_gdf.iterrows():
    ward_id = str(row['WARD'])  # Correct column name
    lon = row.geometry.x
    lat = row.geometry.y
    ward_coords[ward_id] = {'lon': lon, 'lat': lat}

print(f"✅ {len(ward_coords)} ward centroids prepared")
print(f"   Sample ward IDs: {list(ward_coords.keys())[:5]}")

GPM PRECIPITATION FEATURE EXTRACTION

✅ Loaded 141 wards from local file
✅ 141 ward centroids prepared
   Sample ward IDs: ['93', '61', '86', '90', '26']


In [7]:
# GPM IMERG V07 collection
gpm = ee.ImageCollection('NASA/GPM_L3/IMERG_V07')

def calculate_daily_rainfall_batch(date_str):
    """
    Extract daily rainfall for all 141 wards
    Using batch point sampling for speed
    """
    date = datetime.strptime(date_str, '%Y-%m-%d')
    start = date
    end = date + timedelta(days=1)

    # Filter GPM to this day and sum to daily
    daily_precip = gpm.filterDate(
        start.strftime('%Y-%m-%d'),
        end.strftime('%Y-%m-%d')
    ).select('precipitation').sum()

    # Create point FeatureCollection from ward centroids
    points = []
    for ward_id, coords in ward_coords.items():
        point = ee.Geometry.Point([coords['lon'], coords['lat']])
        feature = ee.Feature(point).set('ward_id', ward_id)
        points.append(feature)

    points_fc = ee.FeatureCollection(points)

    # Sample all points at once
    samples = daily_precip.sampleRegions(
        collection=points_fc,
        scale=11000,  # GPM resolution ~11km
        geometries=False
    )

    # Get results
    features = samples.getInfo()['features']

    # Parse to list
    results = []
    for feature in features:
        ward_id = str(feature['properties']['ward_id'])
        rainfall = feature['properties'].get('precipitation', 0)

        results.append({
            'date': date_str,
            'ward_id': ward_id,
            'rain_d0': rainfall if rainfall else 0
        })

    return results

# Test on Sep 20, 2025
print("\n🧪 Testing batch extraction...")
test_results = calculate_daily_rainfall_batch('2025-09-20')

print(f"✅ Batch extraction working")
print(f"   Date: 2025-09-20")
print(f"   Wards processed: {len(test_results)}")
print(f"   Sample rainfall: {test_results[0]['rain_d0']:.2f} mm")


🧪 Testing batch extraction...
✅ Batch extraction working
   Date: 2025-09-20
   Wards processed: 141
   Sample rainfall: 185.89 mm


In [8]:
# Date range: 2014-2025
dates = pd.date_range(start='2014-01-01', end='2025-11-24', freq='D')
date_strings = [d.strftime('%Y-%m-%d') for d in dates]

print(f"\n📅 Processing {len(date_strings):,} dates")
print(f"   From: {date_strings[0]}")
print(f"   To: {date_strings[-1]}")

# Process in batches
batch_size = 30
all_results = []

for i in range(0, len(date_strings), batch_size):
    batch = date_strings[i:i+batch_size]
    batch_num = i//batch_size + 1
    total_batches = len(date_strings)//batch_size + 1

    print(f"\n📦 Batch {batch_num}/{total_batches}")
    print(f"   Dates: {batch[0]} to {batch[-1]}")

    for date_str in batch:
        try:
            daily_data = calculate_daily_rainfall_batch(date_str)
            all_results.extend(daily_data)
        except Exception as e:
            print(f"   ⚠️ Error on {date_str}: {str(e)[:100]}")
            continue

    print(f"   ✅ Completed {len(batch)} dates")
    print(f"   Total rows so far: {len(all_results):,}")

    # Delay between batches
    import time
    time.sleep(2)

# Convert to DataFrame
df_daily = pd.DataFrame(all_results)

print(f"\n✅ ALL DATES PROCESSED")
print(f"   Total rows: {len(df_daily):,}")
print(f"   Unique dates: {df_daily['date'].nunique()}")
print(f"   Unique wards: {df_daily['ward_id'].nunique()}")


📅 Processing 4,346 dates
   From: 2014-01-01
   To: 2025-11-24

📦 Batch 1/145
   Dates: 2014-01-01 to 2014-01-30
   ✅ Completed 30 dates
   Total rows so far: 4,230

📦 Batch 2/145
   Dates: 2014-01-31 to 2014-03-01
   ✅ Completed 30 dates
   Total rows so far: 8,460

📦 Batch 3/145
   Dates: 2014-03-02 to 2014-03-31
   ✅ Completed 30 dates
   Total rows so far: 12,690

📦 Batch 4/145
   Dates: 2014-04-01 to 2014-04-30
   ✅ Completed 30 dates
   Total rows so far: 16,920

📦 Batch 5/145
   Dates: 2014-05-01 to 2014-05-30
   ✅ Completed 30 dates
   Total rows so far: 21,150

📦 Batch 6/145
   Dates: 2014-05-31 to 2014-06-29
   ✅ Completed 30 dates
   Total rows so far: 25,380

📦 Batch 7/145
   Dates: 2014-06-30 to 2014-07-29
   ✅ Completed 30 dates
   Total rows so far: 29,610

📦 Batch 8/145
   Dates: 2014-07-30 to 2014-08-28
   ✅ Completed 30 dates
   Total rows so far: 33,840

📦 Batch 9/145
   Dates: 2014-08-29 to 2014-09-27
   ✅ Completed 30 dates
   Total rows so far: 38,070

📦 Batch 10

In [9]:
# Create date range
dates = pd.date_range(start='2014-01-01', end='2025-11-24', freq='D')
date_strings = [d.strftime('%Y-%m-%d') for d in dates]

print(f"\n📅 Processing {len(date_strings):,} dates")
print(f"   From: {date_strings[0]}")
print(f"   To: {date_strings[-1]}")


📅 Processing 4,346 dates
   From: 2014-01-01
   To: 2025-11-24


In [11]:
# GPM IMERG V07 collection
gpm = ee.ImageCollection('NASA/GPM_L3/IMERG_V07')

def calculate_daily_rainfall_corrected(date_str):
    """
    Extract daily rainfall using point coordinates (NOT Earth Engine assets)
    Uses ward_coords dictionary from cell 4
    """
    date = datetime.strptime(date_str, '%Y-%m-%d')
    start = date
    end = date + timedelta(days=1)

    # Filter GPM to this day
    daily_precip = gpm.filterDate(
        start.strftime('%Y-%m-%d'),
        end.strftime('%Y-%m-%d')
    ).select('precipitation').sum()

    # Create point features from ward_coords dictionary
    points = []
    for ward_id, coords in ward_coords.items():
        point = ee.Geometry.Point([coords['lon'], coords['lat']])
        feature = ee.Feature(point).set('ward_id', ward_id)
        points.append(feature)

    points_fc = ee.FeatureCollection(points)

    # Sample all points at once
    samples = daily_precip.sampleRegions(
        collection=points_fc,
        scale=11000,
        geometries=False
    )

    # Get results
    features = samples.getInfo()['features']

    # Parse
    results = []
    for feature in features:
        ward_id = str(feature['properties']['ward_id'])
        rainfall = feature['properties'].get('precipitation', 0)

        results.append({
            'date': date_str,
            'ward_id': ward_id,
            'rain_d0': rainfall if rainfall else 0
        })

    return results

# Test
print("\n🧪 Testing corrected extraction...")
test_results = calculate_daily_rainfall_corrected('2025-09-20')

print(f"✅ Extraction working")
print(f"   Wards: {len(test_results)}")
print(f"   Sample: Ward {test_results[0]['ward_id']}, {test_results[0]['rain_d0']:.2f} mm")


🧪 Testing corrected extraction...
✅ Extraction working
   Wards: 141
   Sample: Ward 93, 185.89 mm


In [13]:
# ============================================================
# CELL 5: FEATURE ENGINEERING FROM RAW DAILY DATA
# ============================================================

import pandas as pd
import numpy as np

# Pivot to wide format: rows = dates, columns = wards
df_pivot = df_daily.pivot(index='date', columns='ward_id', values='rain_d0')
df_pivot.index = pd.to_datetime(df_pivot.index)
df_pivot = df_pivot.sort_index()

print(f"✅ Pivoted data shape: {df_pivot.shape}")
print(f"   Dates: {df_pivot.index.min()} to {df_pivot.index.max()}")

# Create lag features for each ward
def create_precipitation_features(df_wide):
    """
    Create all precipitation features from wide-format daily data
    Returns long-format DataFrame with all features
    """
    all_features = []

    for ward_id in df_wide.columns:
        ward_series = df_wide[ward_id].copy()

        ward_df = pd.DataFrame({
            'date': ward_series.index,
            'ward_id': ward_id,
            'rain_d0': ward_series.values
        })

        # LAG FEATURES (critical for flood prediction!)
        ward_df['rain_d1'] = ward_series.shift(1).values  # Yesterday
        ward_df['rain_d2'] = ward_series.shift(2).values  # 2 days ago
        ward_df['rain_d3'] = ward_series.shift(3).values  # 3 days ago (Sep 20→23 lag!)

        # ACCUMULATION FEATURES
        ward_df['rain_3day'] = ward_series.rolling(3, min_periods=1).sum().values
        ward_df['rain_7day'] = ward_series.rolling(7, min_periods=1).sum().values
        ward_df['rain_14day'] = ward_series.rolling(14, min_periods=1).sum().values

        # PEAK INDICATORS
        ward_df['max_1day_in_week'] = ward_series.rolling(7, min_periods=1).max().values
        ward_df['rain_7day_avg'] = ward_series.rolling(7, min_periods=1).mean().values

        # TREND FEATURES
        # Days since significant rain (>50mm threshold)
        significant_rain = (ward_series >= 50).astype(int)
        ward_df['days_since_50mm'] = significant_rain.groupby(
            significant_rain.cumsum()
        ).cumcount()

        # Rainfall trend (is it increasing?)
        ward_df['rain_3day_prev'] = ward_series.shift(3).rolling(3, min_periods=1).sum().values
        ward_df['rainfall_increasing'] = (
            ward_df['rain_3day'] > ward_df['rain_3day_prev']
        ).astype(int)

        # INTENSITY INDICATORS
        ward_df['is_heavy_rain'] = (ward_df['rain_d0'] >= 64.5).astype(int)  # IMD heavy rain
        ward_df['is_very_heavy'] = (ward_df['rain_d0'] >= 124.5).astype(int)  # IMD very heavy
        ward_df['is_extreme'] = (ward_df['rain_d0'] >= 244.5).astype(int)  # IMD extreme

        all_features.append(ward_df)

    return pd.concat(all_features, ignore_index=True)

print("\n🔄 Creating precipitation features...")
df_features = create_precipitation_features(df_pivot)

print(f"\n✅ FEATURE ENGINEERING COMPLETE")
print(f"   Total rows: {len(df_features):,}")
print(f"   Features created: {len(df_features.columns)}")
print(f"\n   Feature list:")
for col in df_features.columns:
    print(f"      - {col}")

✅ Pivoted data shape: (4346, 141)
   Dates: 2014-01-01 00:00:00 to 2025-11-24 00:00:00

🔄 Creating precipitation features...

✅ FEATURE ENGINEERING COMPLETE
   Total rows: 612,786
   Features created: 17

   Feature list:
      - date
      - ward_id
      - rain_d0
      - rain_d1
      - rain_d2
      - rain_d3
      - rain_3day
      - rain_7day
      - rain_14day
      - max_1day_in_week
      - rain_7day_avg
      - days_since_50mm
      - rain_3day_prev
      - rainfall_increasing
      - is_heavy_rain
      - is_very_heavy
      - is_extreme


In [14]:
# ============================================================
# CELL 6: VALIDATE SEPTEMBER 2025 FLOOD EVENT
# ============================================================

# The Sep 20→23 lag effect is critical validation
sep_2025 = df_features[
    (df_features['date'] >= '2025-09-18') &
    (df_features['date'] <= '2025-09-25') &
    (df_features['ward_id'] == '1')  # Sample ward
].copy()

print("📊 SEPTEMBER 2025 FLOOD VALIDATION (Ward 1)")
print("=" * 60)
print(sep_2025[['date', 'rain_d0', 'rain_d1', 'rain_d2', 'rain_d3',
                'rain_7day', 'is_heavy_rain']].to_string(index=False))

# Check the lag pattern
sep_20 = df_features[(df_features['date'] == '2025-09-20')]
sep_23 = df_features[(df_features['date'] == '2025-09-23')]

print(f"\n🎯 KEY INSIGHT:")
print(f"   Sep 20 avg rainfall: {sep_20['rain_d0'].mean():.1f} mm (PEAK RAIN)")
print(f"   Sep 23 rain_d3: {sep_23['rain_d3'].mean():.1f} mm (3-day lag captures Sep 20!)")
print(f"   Sep 23 rain_7day: {sep_23['rain_7day'].mean():.1f} mm")

📊 SEPTEMBER 2025 FLOOD VALIDATION (Ward 1)
      date    rain_d0    rain_d1    rain_d2    rain_d3  rain_7day  is_heavy_rain
2025-09-18   1.730000  11.380000  25.990000  34.989999 142.059997              0
2025-09-19   2.540000   1.730000  11.380000  25.990000 124.389997              0
2025-09-20 125.099997   2.540000   1.730000  11.380000 244.199994              1
2025-09-21   8.840000 125.099997   2.540000   1.730000 210.569995              0
2025-09-22  67.019998   8.840000 125.099997   2.540000 242.599994              1
2025-09-23   9.880000  67.019998   8.840000 125.099997 226.489994              0
2025-09-24   5.060000   9.880000  67.019998   8.840000 220.169994              0
2025-09-25  57.089999   5.060000   9.880000  67.019998 275.529993              0

🎯 KEY INSIGHT:
   Sep 20 avg rainfall: 161.3 mm (PEAK RAIN)
   Sep 23 rain_d3: 161.3 mm (3-day lag captures Sep 20!)
   Sep 23 rain_7day: 268.2 mm


In [15]:
# ============================================================
# CELL 7: SAVE FEATURES
# ============================================================

OUTPUT_PATH = Path('../data/processed/precipitation_features_2014_2025.csv')
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)

df_features.to_csv(OUTPUT_PATH, index=False)

print(f"✅ SAVED: {OUTPUT_PATH}")
print(f"   Size: {OUTPUT_PATH.stat().st_size / 1024 / 1024:.1f} MB")
print(f"   Rows: {len(df_features):,}")
print(f"   Date range: {df_features['date'].min()} to {df_features['date'].max()}")

# Summary statistics
print(f"\n📊 FEATURE STATISTICS:")
print(df_features.describe().round(2))

✅ SAVED: ../data/processed/precipitation_features_2014_2025.csv
   Size: 94.8 MB
   Rows: 612,786
   Date range: 2014-01-01 00:00:00 to 2025-11-24 00:00:00

📊 FEATURE STATISTICS:
                      date    rain_d0    rain_d1    rain_d2    rain_d3  \
count               612786  612786.00  612645.00  612504.00  612363.00   
mean   2019-12-13 12:00:00       9.35       9.35       9.36       9.36   
min    2014-01-01 00:00:00       0.00       0.00       0.00       0.00   
25%    2016-12-22 00:00:00       0.00       0.00       0.00       0.00   
50%    2019-12-13 12:00:00       0.00       0.00       0.00       0.00   
75%    2022-12-04 00:00:00       7.38       7.39       7.40       7.40   
max    2025-11-24 00:00:00     315.08     315.08     315.08     315.08   
std                    NaN      22.44      22.45      22.45      22.45   

       rain_3day  rain_7day  rain_14day  max_1day_in_week  rain_7day_avg  \
count  612786.00  612786.00   612786.00         612786.00      612786.00   
me

In [16]:
# ============================================================
# CELL 9: FIX days_since_50mm + VALIDATE SEPTEMBER 2025
# ============================================================

# Fix the days_since_50mm feature
def calculate_days_since_threshold(series, threshold=50):
    """Calculate days since rainfall exceeded threshold"""
    result = []
    days_count = 999  # Start high (no recent heavy rain)

    for val in series:
        if val >= threshold:
            days_count = 0
        else:
            days_count += 1
        result.append(min(days_count, 999))  # Cap at 999

    return result

# Apply fix per ward
df_features = pd.read_csv('../data/processed/precipitation_features_2014_2025.csv')
df_features['date'] = pd.to_datetime(df_features['date'])

fixed_days = []
for ward_id in df_features['ward_id'].unique():
    ward_mask = df_features['ward_id'] == ward_id
    ward_rain = df_features.loc[ward_mask, 'rain_d0'].values
    fixed_days.extend(calculate_days_since_threshold(ward_rain, 50))

df_features['days_since_50mm'] = fixed_days

print("✅ Fixed days_since_50mm")
print(f"   Mean: {df_features['days_since_50mm'].mean():.1f} days")
print(f"   Median: {df_features['days_since_50mm'].median():.1f} days")

# ============================================================
# SEPTEMBER 2025 VALIDATION
# ============================================================

print("\n" + "=" * 60)
print("📊 SEPTEMBER 2025 FLOOD VALIDATION")
print("=" * 60)

# Get data for one ward during the flood
ward_1_sep = df_features[
    (df_features['date'] >= '2025-09-18') &
    (df_features['date'] <= '2025-09-25') &
    (df_features['ward_id'] == '1')
][['date', 'rain_d0', 'rain_d1', 'rain_d2', 'rain_d3', 'rain_7day', 'is_heavy_rain']]

print("\nWard 1 - Daily Pattern:")
print(ward_1_sep.to_string(index=False))

# City-wide averages
print("\n🎯 CITY-WIDE AVERAGES:")
for date_str in ['2025-09-20', '2025-09-21', '2025-09-22', '2025-09-23']:
    day_data = df_features[df_features['date'] == date_str]
    print(f"\n{date_str}:")
    print(f"   rain_d0: {day_data['rain_d0'].mean():6.1f} mm (same day)")
    print(f"   rain_d3: {day_data['rain_d3'].mean():6.1f} mm (3-day lag)")
    print(f"   rain_7day: {day_data['rain_7day'].mean():6.1f} mm (weekly accum)")

# Re-save with fix
df_features.to_csv('../data/processed/precipitation_features_2014_2025.csv', index=False)
print("\n✅ Re-saved with fixed days_since_50mm")

✅ Fixed days_since_50mm
   Mean: 61.5 days
   Median: 24.0 days

📊 SEPTEMBER 2025 FLOOD VALIDATION

Ward 1 - Daily Pattern:
Empty DataFrame
Columns: [date, rain_d0, rain_d1, rain_d2, rain_d3, rain_7day, is_heavy_rain]
Index: []

🎯 CITY-WIDE AVERAGES:

2025-09-20:
   rain_d0:  161.3 mm (same day)
   rain_d3:   10.8 mm (3-day lag)
   rain_7day:  285.5 mm (weekly accum)

2025-09-21:
   rain_d0:    8.2 mm (same day)
   rain_d3:    1.1 mm (3-day lag)
   rain_7day:  254.1 mm (weekly accum)

2025-09-22:
   rain_d0:   69.8 mm (same day)
   rain_d3:    5.9 mm (3-day lag)
   rain_7day:  285.5 mm (weekly accum)

2025-09-23:
   rain_d0:   11.1 mm (same day)
   rain_d3:  161.3 mm (3-day lag)
   rain_7day:  268.2 mm (weekly accum)

✅ Re-saved with fixed days_since_50mm
